In [1]:
#%reset
import sys
sys.path.append('/usr/local/lib/python2.7/site-packages')
sys.path.append('./modules')

import warnings
#import re

import pandas as pd
#import numpy as np
from sklearn import metrics

## import modules to build pipelines
import pipemodules as pm
import projecthandle as proj
import run_grid as rg

#% matplotlib inline

In [2]:
## Read csv file containing descriptors
descriptors_raw = pd.read_csv('./rdkit_descriptors.csv')
labels = descriptors_raw.iloc[:,0]
X = descriptors_raw.iloc[:,2:]
y = descriptors_raw.iloc[:,1]

del descriptors_raw

#range(10,100,10),
#,2,3,4,6,7,8,9,10,11,12])

In [8]:
import pickle
import pipemodules as pm
import pandas as pd


results.reset_index(drop=True, inplace=True)

## filter results and eliminate poor models
for i in range(0,len(results)):
    if results.mean_train_score[i] > 0.75 and abs(results.mean_test_score[i] - results.mean_train_score[i]) < 0.15:
        continue
    else: 
        results.drop(i, axis=0, inplace=True)

results.reset_index(drop=True, inplace=True)

## create analysis set
# set arrays for results
dev_set_score = []
eval_set_score = []
dev_evs = []
eval_evs = []
dev_mae = []
eval_mae = []
dev_mse = []
eval_mse = []
dev_medae = []
eval_medae = []
method_ids = []
parameters = []


for i in range(0,len(results)):
    ## take method_ids and build estimator for current method
    string = results.method_ids[i] # retrive method id
    setup = eval(string) # convert to iterable array

    temp = pm.search_random_forest() #initiate class

    # set the estimator type and initiate estimator class
    _,clf,_ = temp.set_method(setup[2]) 
    
    # get the development set features
    X_dev_temp, _ = pm.get_X(dev_set.matrix_raw, \
                             meth.indvals[setup[0]][setup[1]]) 
    # get the evaluation set features
    X_eval_temp, _ = pm.get_X(eval_set.matrix_raw, \
                              meth.indvals[setup[0]][setup[1]]) 

    del temp

    # retreive hyper-parameters
    params = results['params'][i]
    # set estimator hyper-parameters
    clf.set_params(**params)
    
    # fit the estimator to the development set
    clf.fit(X_dev_temp, dev_set.y_raw)
    # predict the evaluation set
    eval_predict = clf.predict(X_eval_temp)
    # predict the development set - for metrics
    dev_predict = clf.predict(X_dev_temp)
    
    # add calculated metrics, methods, and parameters to lists for results
    dev_set_score.append(clf.score(X_dev_temp, dev_set.y_raw))
    eval_set_score.append(clf.score(X_eval_temp, eval_set.y_raw))
    dev_evs.append(metrics.explained_variance_score(dev_predict, dev_set.y_raw))
    eval_evs.append(metrics.explained_variance_score(eval_predict, eval_set.y_raw))
    dev_mae.append(metrics.mean_absolute_error(dev_predict, dev_set.y_raw))
    eval_mae.append(metrics.mean_absolute_error(eval_predict, eval_set.y_raw))
    dev_mse.append(metrics.mean_squared_error(dev_predict, dev_set.y_raw))
    eval_mse.append(metrics.mean_squared_error(eval_predict, eval_set.y_raw))
    dev_medae.append(metrics.median_absolute_error(dev_predict, dev_set.y_raw))
    eval_medae.append(metrics.median_absolute_error(eval_predict, eval_set.y_raw))
    method_ids.append(string)
    parameters.append(params)
    
# create dictionary object from results
evaluation_results = {'dev_set_score':dev_set_score, 'eval_set_score':eval_set_score, \
                     'method_ids':method_ids, 'parameters':parameters, 'dev_evs':dev_evs, \
                     'eval_evs':eval_evs, 'dev_mae':dev_mae, 'eval_mae':eval_mae, \
                     'dev_mse': dev_mse, 'eval_mse':eval_mse, 'dev_median_ae':dev_medae, \
                     'eval_median_ae':eval_medae}

# re-rank and sort filtered methods by test-score (r**2)
analysis_set = pd.DataFrame(evaluation_results)
array = np.array(analysis_set['eval_set_score'])
temp = array.argsort()[::-1]
ranks = np.empty(len(array), int)
ranks[temp] = np.arange(len(array))
analysis_set['rank_test_score'] = ranks
analysis_set.sort_values(by='rank_test_score', inplace=True)
analysis_set.reset_index(drop=True, inplace=True)
analysis_set.to_dictionary()

/Users/res3/Library/Python/2.7/lib/python/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 12 iterations, i.e. alpha=1.094e-02, with an active set of 8 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/Users/res3/Library/Python/2.7/lib/python/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 20 iterations, i.e. alpha=5.470e-03, with an active set of 14 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
/Users/res3/Library/Python/2.7/lib/python/site-packages/sklearn/linear_model/least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 23 iterations, i.e. alpha=3.718e-03, with an active set of 17 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)

In [3]:
eval_set = rg.auto_grid(X,y,labels, ks=[10,20], opts=[1,2])

   mean_fit_time  mean_score_time  mean_test_score  mean_train_score  \
0       0.024136         0.005008         0.473376          0.883712   
1       0.060178         0.011970         0.492589          0.913149   
2       0.095356         0.018556         0.501884          0.919258   
3       0.131009         0.025559         0.495889          0.920909   
4       0.050747         0.005121         0.485578          0.894911   
5       0.135085         0.011601         0.484008          0.915978   
6       0.215881         0.022047         0.491211          0.916460   
7       0.287532         0.025424         0.499552          0.918961   

  method_ids param_criterion param_n_estimators  \
0  [0, 0, 1]             mse                 10   
1  [0, 0, 1]             mse                 25   
2  [0, 0, 1]             mse                 40   
3  [0, 0, 1]             mse                 55   
4  [0, 0, 1]             mae                 10   
5  [0, 0, 1]             mae                 

In [53]:
currind = 0
to_drop = []
for i in range(0, len(evaluation_results)-1):
    if abs(evaluation_results.eval_set_score[currind] - evaluation_results.eval_set_score[i+1]) < 0.01 \
        and abs(evaluation_results.dev_set_score[i] - evaluation_results.dev_set_score[i+1]) < 0.01:
        to_drop.append(i+1)
    else:
        currind = i


In [54]:
for i in to_drop:
    evaluation_results.drop(i, inplace=True)
    
evaluation_results.sort_values(by='rank_test_score', inplace=True)
evaluation_results.reset_index(drop=True, inplace=True)

In [55]:
to_drop = []
for i in range(0, len(evaluation_results)-1):
    if str(evaluation_results.method_ids[i])==str(evaluation_results.method_ids[i+1]):
        to_drop.append(i+1)
    else:
        continue

for i in to_drop:
    evaluation_results.drop(i, inplace=True)

evaluation_results.sort_values(by='rank_test_score', inplace=True)
evaluation_results.reset_index(drop=True, inplace=True)

#array = np.array(evaluation_results['eval_set_score'])
#temp = array.argsort()[::-1]
#ranks = np.empty(len(array), int)
#ranks[temp] = np.arange(len(array))
#evaluation_results['rank_test_score'] = ranks

In [143]:
save_analysis(evaluation_results, './results.p')

This file contains an evaluation and analysis set


IOError: Trying to write to wrong type of file, or analysis set already saved! Check the file...

In [5]:
eval_set.results


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,method_ids,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.459420,0.029228,0.636866,0.999327,"[1, 2, 2]","{u'n_estimators': 55, u'criterion': u'mae'}",0,0.677133,0.999978,0.666931,0.998030,0.566100,0.999974,0.016674,0.000592,0.050079,0.000917
1,0.335544,0.019676,0.621732,0.999327,"[1, 2, 2]","{u'n_estimators': 40, u'criterion': u'mae'}",1,0.667524,0.999978,0.648800,0.998030,0.548378,0.999974,0.010716,0.000828,0.052292,0.000917
2,0.139993,0.027920,0.619020,0.999327,"[1, 2, 2]","{u'n_estimators': 55, u'criterion': u'mse'}",2,0.668569,0.999978,0.638537,0.998030,0.549420,0.999974,0.004264,0.001138,0.050593,0.000917
3,0.439856,0.027308,0.618587,0.940927,"[1, 2, 1]","{u'n_estimators': 55, u'criterion': u'mae'}",3,0.662526,0.932603,0.615139,0.945497,0.577623,0.944681,0.019597,0.000795,0.034777,0.005895
4,0.156221,0.028420,0.617331,0.942951,"[1, 2, 1]","{u'n_estimators': 55, u'criterion': u'mse'}",4,0.715424,0.934945,0.579221,0.947193,0.556294,0.946716,0.006242,0.001802,0.070357,0.005665
5,0.123745,0.021479,0.615974,0.941097,"[1, 2, 1]","{u'n_estimators': 40, u'criterion': u'mse'}",5,0.693874,0.929307,0.626593,0.941446,0.526619,0.952537,0.006429,0.003365,0.068751,0.009487
6,0.103084,0.021112,0.610592,0.999327,"[1, 2, 2]","{u'n_estimators': 40, u'criterion': u'mse'}",6,0.675135,0.999978,0.610547,0.998030,0.545401,0.999974,0.003110,0.000726,0.053010,0.000917
7,0.199907,0.014017,0.608719,0.934413,"[1, 2, 1]","{u'n_estimators': 25, u'criterion': u'mae'}",7,0.635584,0.933998,0.660207,0.920911,0.530077,0.948331,0.012959,0.000620,0.056365,0.011198
8,0.084314,0.005619,0.598731,0.999327,"[1, 2, 2]","{u'n_estimators': 10, u'criterion': u'mae'}",8,0.639206,0.999978,0.601776,0.998030,0.554777,0.999974,0.003891,0.000318,0.034565,0.000917
9,0.068220,0.012536,0.593994,0.999327,"[1, 2, 2]","{u'n_estimators': 25, u'criterion': u'mse'}",9,0.674517,0.999978,0.600672,0.998030,0.505927,0.999974,0.001923,0.000309,0.069048,0.000917


In [3]:
stuff.load_file('./results.p')

This file contains only an evaluation set
This file contains an evaluation and analysis set
5
hi
hi


In [4]:
stuff.analysis_results

,dev_evs,dev_mae,dev_median_ae,dev_mse,dev_set_score,eval_evs,eval_mae,eval_median_ae,eval_mse,eval_set_score,method_ids,parameters,rank_test_score
0,0.739102,0.623838,0.501034,0.639926,0.819417,0.694692,0.616625,0.441952,0.720915,0.787241,"[7, 0, 11]","{u'normalize': True, u'l1_ratio': 0.8, u'n_alp...",0
1,0.744669,0.617112,0.483138,0.626944,0.823081,0.699515,0.612576,0.466975,0.721839,0.786968,"[8, 0, 11]","{u'normalize': True, u'l1_ratio': 0.8, u'n_alp...",13
2,0.733978,0.636807,0.511334,0.669066,0.811194,0.710737,0.615025,0.434259,0.721846,0.786966,"[5, 2, 11]","{u'normalize': True, u'l1_ratio': 0.8, u'n_alp...",14
3,0.745762,0.616421,0.480062,0.625489,0.823491,0.700215,0.612471,0.465889,0.721890,0.786953,"[8, 0, 11]","{u'normalize': True, u'l1_ratio': 0.8, u'n_alp...",18
4,0.733186,0.637282,0.511426,0.669930,0.810950,0.710194,0.615207,0.435337,0.721912,0.786947,"[5, 2, 11]","{u'normalize': True, u'l1_ratio': 0.8, u'n_alp...",19
5,0.748417,0.614667,0.475049,0.621882,0.824509,0.701933,0.612238,0.459401,0.721978,0.786927,"[8, 0, 11]","{u'normalize': True, u'l1_ratio': 0.8, u'n_alp...",21
6,0.732047,0.637962,0.511549,0.671181,0.810597,0.709410,0.615465,0.436924,0.722020,0.786915,"[5, 2, 11]","{u'normalize': True, u'l1_ratio': 0.8, u'n_alp...",29
7,0.694322,0.657794,0.518539,0.714293,0.798431,0.684489,0.627613,0.460684,0.736574,0.782620,"[3, 2, 11]","{u'normalize': True, u'l1_ratio': 0.8, u'n_alp...",76
8,0.752357,0.632101,0.494662,0.648956,0.816869,0.711772,0.618592,0.453992,0.738298,0.782111,"[5, 2, 11]","{u'normalize': True, u'l1_ratio': 0.4, u'n_alp...",77
9,0.732740,0.638443,0.486457,0.673809,0.809855,0.706970,0.624094,0.438201,0.738805,0.781961,"[4, 2, 11]","{u'normalize': True, u'l1_ratio': 0.8, u'n_alp...",96


In [5]:
all_data

NameError: name 'all_data' is not defined